In [1]:
import cv2
import os
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
from mtcnn import MTCNN

In [2]:
test_sample = '../input/deepfake-detection-challenge/test_videos/'
test_video_files = [test_sample + x for x in sorted(os.listdir(test_sample))]

In [6]:
try:
    os.makedirs('../working/dataset/TEST3')
except:
    pass

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [8]:
train_video_files = sorted(original_vidio_has_fake)
train_video_files

NameError: name 'original_vidio_has_fake' is not defined

In [18]:
# origin 영상들로 학습시작
#detector = dlib.get_frontal_face_detector()
train_video_files = sorted(original_vidio_has_fake)

#train_video_files = ['atvmxvwyns.mp4']

mtcnn = MTCNN() # device='cuda:0',

vid_num = 0
all_vid = len(train_video_files)
# origin 영상 얼굴찾기
for vid in train_video_files:
    count = 0
    path = '../input/deepfake-detection-challenge/train_sample_videos/' + vid
    file_name = vid.split('.')[0]
    face_coord = {}
    cap = cv2.VideoCapture(path)
    frame = 0

    
    before_face_img_coord = []
    
    while(cap.isOpened()):
        ret = cap.grab()
        if ret == False:
            break
        #if frame % 30 == 0:
        ret, image = cap.retrieve()
        #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img = image
        face_coord[frame]=[]
        # 처음 얼굴이 나오면
        if not before_face_img_coord:
            faces = mtcnn.detect_faces(img)
            # 얼굴 한개 이상?
            for face_idx, face in enumerate(faces):
                if 'confidence' in face and face['confidence'] > 0.9:
                    x1,y1,w,h = face['box']
                    x2 = min(x1+w, img.shape[1])
                    y2 = min(y1+h, img.shape[0])
                    x1 = max(x1, 0)
                    y1 = max(y1, 0)
                    crop_img = img[y1:y2, x1:x2]
                    face_coord[frame].append([y1,y2,x1,x2])

#                     plt.imshow(crop_img)
#                     plt.show()
#                     print(frame)
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img, (160, 160)))
                    before_face_img_coord.append([y1,y2,x1,x2])
                    
                    break
                    
        # 이전에 검출된 face가 있으면
        else:
            for face_idx, coord in enumerate(before_face_img_coord):
                im = img[max(coord[0]-50,0) : min(coord[1]+50,img.shape[0]),
                         max(coord[2]-50,0) : min(coord[3]+50,img.shape[1])]
                face = mtcnn.detect_faces(im)
                if face:
                    # 1개만 저장해
                    x11,y11,w,h = face[0]['box']
                    x1 = max(max(coord[2]-50, 0) + x11, 0)
                    x2 = min(x1 + w, img.shape[1])
                    y1 = max(max(coord[0]-50, 0) + y11, 0)
                    y2 = min(y1 + h, img.shape[0])
                    crop_img2 = img[y1:y2, x1:x2]
                    
                    
                    face_coord[frame].append([y1,y2,x1,x2])
                    before_face_img_coord[face_idx]=[y1,y2,x1,x2]
                    
#                     plt.imshow(crop_img2)
#                     plt.show()
#                     print(frame)
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img2, (160, 160)))
                    crop_img = crop_img2
                else:
                    #face_coord[frame].append([y1,y2,x1,x2])
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img, (160, 160)))

        frame += 1
        if not face_coord[0]:
            break
    cap.release()


    
    # 이비디오의 fake 영상만큼 반복
    for fake_vid in train_sample_metadata[train_sample_metadata['original']==vid].index:
        if not face_coord[0]:
            break
        fake_path = '../input/deepfake-detection-challenge/train_sample_videos/' + fake_vid
        cap2 = cv2.VideoCapture(fake_path)
        fake_vid_name = fake_vid.split('.')[0]
        frame2 = 0
        while(cap2.isOpened()):
            ret2 = cap2.grab()
            if ret2 == False:
                break
            #if frame2 % 30 == 0:
            ret2, image2 = cap2.retrieve()
            #img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
            if face_coord[frame2]:
                for face_idx, i in enumerate(face_coord[frame2]):
                    y1,y2,x1,x2 = i
                    crop_img3 = image2[y1:y2, x1:x2]
                    cv2.imwrite(f'../working/dataset/FAKE3/FAKE_{fake_vid_name}_{frame2}_{face_idx}_{file_name}.jpg', cv2.resize(crop_img3, (160, 160)))
            else:
#                 print(fake_vid, frame2)
#                 plt.imshow(crop_img3)
#                 plt.show()
                cv2.imwrite(f'../working/dataset/FAKE3/FAKE_{fake_vid_name}_{frame2}_{face_idx}_{file_name}.jpg', cv2.resize(crop_img3, (160, 160)))
            frame2 += 1
        cap2.release()

        
    print(f'{vid_num}/{all_vid}')
    vid_num += 1
    

0/42
1/42
2/42
3/42
4/42
5/42
6/42
7/42
8/42
9/42
10/42
11/42
12/42
13/42
14/42
15/42
16/42
17/42
18/42
19/42
20/42
21/42
22/42
23/42
24/42
25/42
26/42
27/42
28/42
29/42
30/42
31/42
32/42
33/42
34/42
35/42
36/42
37/42
38/42
39/42
40/42
41/42
